In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tweepy
import pandas as pd
import numpy as np
import random 
import json
import tweepy 

random.seed(416)
np.random.seed(416)

with open('twitter_creds3.json', 'r') as file:
    secrets = json.load(file)
TWITTER_API = secrets['personal_news']

# Read in data

In [2]:
dfs = []
usernames = ['charliekirk11', 'gatewaypundit', 'jackposobiec', 'realcandaceo', 'stkirsch']
conditions = ['treat', 'ctrl']

for username in usernames:
    for condition in conditions:
        df = pd.read_csv(f'{condition}_twit_{username}.txt', header=None)
        df['condition'] = condition
        df['username'] = username
        df.columns = ['follower_id', 'condition', 'spreader_username']
        dfs.append(df)
    
master_df = pd.concat(dfs)

# Hydrate Ids

In [3]:
panel_n = 200 # how many followers we want per (spreader, condition) block 
panel_n_pad = 50 # how many followers we want to pad each block with....assuming some will not be reachable when we try to hydrate ids
final_panel_n = panel_n + panel_n_pad # total number of followers we want to hydrate per block
panel = master_df.groupby(['spreader_username', 'condition']).apply(lambda x: x.sample(n=panel_n, replace=False)).reset_index(drop=True)


/var/folders/kj/8p_kqzx533b8vldhm5sdjfc00000gn/T/ipykernel_6914/3765090130.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  panel = master_df.groupby(['spreader_username', 'condition']).apply(lambda x: x.sample(n=panel_n, replace=False)).reset_index(drop=True)


In [24]:
panel

,follower_id,condition,spreader_username
0,1612249376176365570,ctrl,charliekirk11
1,493332624,ctrl,charliekirk11
2,1724168354418601984,ctrl,charliekirk11
3,2318621556,ctrl,charliekirk11
4,1499417623410446336,ctrl,charliekirk11
...,...,...,...
1995,136179128,treat,stkirsch
1996,1236859643478982656,treat,stkirsch
1997,1258588813296713729,treat,stkirsch
1998,1487878309656481793,treat,stkirsch


# Scrap

In [4]:
client = tweepy.Client(bearer_token=TWITTER_API['bearer_token'], wait_on_rate_limit=True)

In [231]:
tweets_response = client.get_users_tweets(
    id='1481596104152125442',
    max_results=5,
    tweet_fields=[
        "attachments", "author_id", "conversation_id",
        "created_at", "entities", "geo", "id", "in_reply_to_user_id", "lang",  "public_metrics", "referenced_tweets", "reply_settings", 
        "source", "text", "withheld", "note_tweet"
    ],
    media_fields=['url', 'preview_image_url'],
    expansions=[
        "attachments.poll_ids", "attachments.media_keys", "author_id", "geo.place_id", 
        "in_reply_to_user_id", "referenced_tweets.id", "entities.mentions.username", 
        "referenced_tweets.id.author_id", 
    ]
)



In [232]:
tweet = tweets_response.data[0].data
tweet

{'edit_history_tweet_ids': ['1775208072459555070'],
 'conversation_id': '1775208072459555070',
 'reply_settings': 'everyone',
 'author_id': '1481596104152125442',
 'entities': {'urls': [{'start': 0,
    'end': 23,
    'url': 'https://t.co/ntN87xYrX2',
    'expanded_url': 'https://www.nytimes.com/interactive/2024/03/28/opinion/ai-political-bias.html',
    'display_url': 'nytimes.com/interactive/20…',
    'images': [{'url': 'https://pbs.twimg.com/news_img/1773276550726647809/yuRFxxWN?format=jpg&name=orig',
      'width': 1600,
      'height': 900},
     {'url': 'https://pbs.twimg.com/news_img/1773276550726647809/yuRFxxWN?format=jpg&name=150x150',
      'width': 150,
      'height': 150}],
    'status': 200,
    'title': 'Opinion | How A.I. Chatbots Become Political',
    'description': 'We may soon rely a lot on A.I. chatbots, so keeping an eye on their political leanings is becoming more and more important.',
    'unwound_url': 'https://www.nytimes.com/interactive/2024/03/28/opinion/ai-

# Working messy

In [228]:

def parse_tweet(tweet, includes):
    includes_tweet_data = [tweets_response.includes['tweets'][i].data for i in range(len(tweets_response.includes['tweets']))]
    includes_user_data = [tweets_response.includes['users'][i].data for i in range(len(tweets_response.includes['users']))]

    urls = tweet.get('entities', {}).get('urls', []) if 'entities' in tweet else []
    
    if urls:
        tweet['primary_urls'] = [x['unwound_url'] for x in urls]
    else:
        tweet['primary_urls'] = []
        pass

    if 'referenced_tweet' not in tweet:
        tweet['referenced_tweets'] = []
    else:
        ref_tweets = tweet['referenced_tweets']
        for ref_tweet in ref_tweets:
            ref_tweet['urls'] = []
            ref_tweet['ref_author_id'] = None
            ref_tweet['ref_author_username'] = None
    
            rel_tweet = ref_tweet['type']
            tweet_id = ref_tweet['id']
    
            # Get tweet data from the retweet 
            for t in includes_tweet_data:
                if t['id'] == tweet_id:
                    try:
                        urls = t.get('entities', {}).get('urls', []) if 'entities' in t else []
                        expanded_urls = [url['unwound_url'] for url in urls if 'unwound_url' in url]
                        ref_tweet['urls'].extend(expanded_urls)
                        ref_tweet['ref_author_id'] = t['author_id']
                    except:
                        pass
    
            # Get author data from the retweet
            for t in includes_user_data:
                if t['id']== ref_tweet['ref_author_id']:
                    ref_tweet['ref_author_username'] = t['username']
                    
    return tweet
        


data = []
for i in range(len(tweets_response.data)):
    data.append(parse_tweet(tweets_response.data[i].data , tweets_response.includes))




In [230]:
data[0]

{'entities': {'urls': [{'start': 0,
    'end': 23,
    'url': 'https://t.co/ntN87xYrX2',
    'expanded_url': 'https://www.nytimes.com/interactive/2024/03/28/opinion/ai-political-bias.html',
    'display_url': 'nytimes.com/interactive/20…',
    'images': [{'url': 'https://pbs.twimg.com/news_img/1773276550726647809/yuRFxxWN?format=jpg&name=orig',
      'width': 1600,
      'height': 900},
     {'url': 'https://pbs.twimg.com/news_img/1773276550726647809/yuRFxxWN?format=jpg&name=150x150',
      'width': 150,
      'height': 150}],
    'status': 200,
    'title': 'Opinion | How A.I. Chatbots Become Political',
    'description': 'We may soon rely a lot on A.I. chatbots, so keeping an eye on their political leanings is becoming more and more important.',
    'unwound_url': 'https://www.nytimes.com/interactive/2024/03/28/opinion/ai-political-bias.html'}]},
 'public_metrics': {'retweet_count': 0,
  'reply_count': 0,
  'like_count': 0,
  'quote_count': 0,
  'bookmark_count': 0,
  'impression_co

In [194]:
tweet

{'public_metrics': {'retweet_count': 6,
  'reply_count': 0,
  'like_count': 0,
  'quote_count': 0,
  'bookmark_count': 0,
  'impression_count': 0},
 'edit_history_tweet_ids': ['1766181347406798927'],
 'text': 'RT @boixserra: Polarization (and public opinion evaluation of whether it can be overcome) across the world. \nVia @politico: https://t.co/lO…',
 'author_id': '1481596104152125442',
 'entities': {'mentions': [{'start': 3,
    'end': 13,
    'username': 'boixserra',
    'id': '471649147'},
   {'start': 113, 'end': 122, 'username': 'politico', 'id': '9300262'}]},
 'id': '1766181347406798927',
 'created_at': '2024-03-08T19:17:01.000Z',
 'reply_settings': 'everyone',
 'conversation_id': '1766181347406798927',
 'referenced_tweets': [{'type': 'retweeted', 'id': '1753054539396075900'}],
 'lang': 'en',
 'edit_controls': {'edits_remaining': 5,
  'is_edit_eligible': False,
  'editable_until': datetime.datetime(2024, 3, 8, 20, 17, 1, tzinfo=datetime.timezone.utc)}}

In [73]:
tweet_data = [tweets_response.data[i].data for i in range(len(tweets_response.data))]
includes_user_data = [tweets_response.includes['users'][i].data for i in range(len(tweets_response.includes['users']))]
includes_tweet_data = [tweets_response.includes['tweets'][i].data for i in range(len(tweets_response.includes['tweets']))]

# d = {'1481596104152125442':{'data':[tweets_response.data[i].data for i in range(len(tweets_response.data))]}
#      'includes':tweets_response.includes[i].data for i in range(len(tweets_response.data
#     }


In [81]:
includes_tweet_data[2]

{'text': 'RT @umsi: Spotlight on two new research papers led by UMSI PhD student Joshua Ashkinaze 💡 @JoshAshkinaze\n\nHow AI Ideas Affect the Creativit…',
 'reply_settings': 'everyone',
 'referenced_tweets': [{'type': 'retweeted', 'id': '1766111764263870608'}],
 'lang': 'en',
 'edit_controls': {'edits_remaining': 5,
  'is_edit_eligible': False,
  'editable_until': datetime.datetime(2024, 3, 8, 20, 4, 37, tzinfo=datetime.timezone.utc)},
 'author_id': '1481596104152125442',
 'context_annotations': [{'domain': {'id': '30',
    'name': 'Entities [Entity Service]',
    'description': 'Entity Service top level domain, every item that is in Entity Service should be in this domain'},
   'entity': {'id': '781974596752842752', 'name': 'Services'}},
  {'domain': {'id': '46',
    'name': 'Business Taxonomy',
    'description': 'Categories within Brand Verticals that narrow down the scope of Brands'},
   'entity': {'id': '1552291412820893697',
    'name': 'Education Business',
    'description': 'B

In [141]:
l.data

{'reply_settings': 'everyone',
 'entities': {'mentions': [{'start': 0,
    'end': 14,
    'username': 'JustinTrudeau',
    'id': '14260960'}],
  'annotations': [{'start': 20,
    'end': 24,
    'probability': 0.352,
    'type': 'Person',
    'normalized_text': 'Moron'}]},
 'created_at': '2024-03-28T06:44:12.000Z',
 'in_reply_to_user_id': '14260960',
 'edit_history_tweet_ids': ['1773239648468295940'],
 'author_id': '1612249376176365570',
 'public_metrics': {'retweet_count': 0,
  'reply_count': 0,
  'like_count': 0,
  'quote_count': 0,
  'bookmark_count': 0,
  'impression_count': 2},
 'text': '@JustinTrudeau This Moron is all coked out and drops this post at midnight Do another line off of your desk in your office \nGo back to sleep blackface and resign in disgrace \nNo one believes your lies anymore because you caused this problem with all your policies and taxation to death',
 'id': '1773239648468295940',
 'edit_controls': {'edits_remaining': 5,
  'is_edit_eligible': False,
  'editable

In [200]:
includes

{'users': [<User id=1481596104152125442 name=Joshua Ashkinaze username=JoshAshkinaze>,
  <User id=471649147 name=Carles Boix 🇺🇦 username=boixserra>,
  <User id=9300262 name=POLITICO username=politico>,
  <User id=18033550 name=School of Information username=umsi>,
  <User id=15184929 name=Ross Dawson username=rossdawson>],
 'tweets': [<Tweet id=1766181347406798927 text='RT @boixserra: Polarization (and public opinion evaluation of whether it can be overcome) across the world. \nVia @politico: https://t.co/lO…'>,
  <Tweet id=1753054539396075900 text='Polarization (and public opinion evaluation of whether it can be overcome) across the world. \nVia @politico: https://t.co/lOBTMouIbx https://t.co/MVOqCDYIDE'>,
  <Tweet id=1766178225716343208 text='RT @umsi: Spotlight on two new research papers led by UMSI PhD student Joshua Ashkinaze 💡 @JoshAshkinaze\n\nHow AI Ideas Affect the Creativit…'>,
  <Tweet id=1766111764263870608 text='Spotlight on two new research papers led by UMSI PhD student 

In [ ]:
1753054539396075900

In [151]:
def get_data(user_id):


resp = client.get_tweet(id='1753054539396075900', 
                                
                                          tweet_fields=['entities', 'referenced_tweets', 'public_metrics', 'attachments', 'source'],
                                          media_fields = ['url', 'preview_image_url'],
                                         expansions=['attachments.media_keys', 
                                                     'entities.mentions.username', 
                                                     'in_reply_to_user_id', 
                                                     'author_id', 
                                                    'referenced_tweets.id', 
    

]
                                         ) 


In [157]:
urls = a.data.data['entities']['urls']

In [159]:
urls[0]

{'start': 109,
 'end': 132,
 'url': 'https://t.co/lOBTMouIbx',
 'expanded_url': 'https://www.politico.eu/newsletter/digital-bridge/how-the-west-was-radicalized/',
 'display_url': 'politico.eu/newsletter/dig…',
 'images': [{'url': 'https://pbs.twimg.com/news_img/1756302931819655168/Sh2lC_rJ?format=jpg&name=orig',
   'width': 1600,
   'height': 900},
  {'url': 'https://pbs.twimg.com/news_img/1756302931819655168/Sh2lC_rJ?format=jpg&name=150x150',
   'width': 150,
   'height': 150}],
 'status': 200,
 'title': 'How the West was radicalized',
 'description': "POLITICO's weekly transatlantic tech newsletter uncovers the digital relationship between critical power-centers through exclusive insights and breaking news for global technology elites and political influencers.",
 'unwound_url': 'https://www.politico.eu/newsletter/digital-bridge/how-the-west-was-radicalized/'}

In [ ]:
def parse_urls(url_dict):
    data = []
    for url_info in urls:
        data.append(
        

In [208]:
includes['users'][0].data

{'id': '1481596104152125442',
 'name': 'Joshua Ashkinaze',
 'username': 'JoshAshkinaze'}

In [9]:
tweets_response

Response(data=[<Tweet id=1766181347406798927 text='RT @boixserra: Polarization (and public opinion evaluation of whether it can be overcome) across the world. \nVia @politico: https://t.co/lO…'>, <Tweet id=1766178225716343208 text='RT @umsi: Spotlight on two new research papers led by UMSI PhD student Joshua Ashkinaze 💡 @JoshAshkinaze\n\nHow AI Ideas Affect the Creativit…'>, <Tweet id=1765258076662214795 text='RT @umsi: New research from UMSI:\nThe Dynamics of (Not) Unfollowing Misinformation Spreaders\nJoshua Ashkinaze, Eric Gilbert, Ceren Budak\n@e…'>, <Tweet id=1765257663166714275 text='RT @umsi: New research from UMSI:\nHow AI Ideas Affect the Creativity, Diversity, and Evolution of Human Ideas: Evidence From a Large, Dynam…'>, <Tweet id=1765257323616886903 text='RT @rossdawson: Distilled from “How AI Ideas Affect the Creativity, Diversity, and Evolution of Human Ideas: Evidence From a Large, Dynamic…'>], includes={'users': [<User id=1481596104152125442 name=Joshua Ashkinaze userna

In [21]:
data = tweets_response.data
includes = tweets_response.includes
t = data[0].data
t


{'text': 'RT @boixserra: Polarization (and public opinion evaluation of whether it can be overcome) across the world. \nVia @politico: https://t.co/lO…',
 'reply_settings': 'everyone',
 'referenced_tweets': [{'type': 'retweeted', 'id': '1753054539396075900'}],
 'lang': 'en',
 'edit_controls': {'edits_remaining': 5,
  'is_edit_eligible': False,
  'editable_until': datetime.datetime(2024, 3, 8, 20, 17, 1, tzinfo=datetime.timezone.utc)},
 'author_id': '1481596104152125442',
 'id': '1766181347406798927',
 'edit_history_tweet_ids': ['1766181347406798927'],
 'entities': {'mentions': [{'start': 3,
    'end': 13,
    'username': 'boixserra',
    'id': '471649147'},
   {'start': 113, 'end': 122, 'username': 'politico', 'id': '9300262'}]},
 'public_metrics': {'retweet_count': 6,
  'reply_count': 0,
  'like_count': 0,
  'quote_count': 0,
  'bookmark_count': 0,
  'impression_count': 0},
 'conversation_id': '1766181347406798927',
 'created_at': '2024-03-08T19:17:01.000Z'}

In [18]:
if 'referenced_tweets' in t:
    for ref_tweet in t['referenced_tweets']:
        print(ref_tweet)

{'type': 'retweeted', 'id': '1753054539396075900'}


In [26]:
for x in includes['tweets']:
    print(x.data)

{'text': 'RT @boixserra: Polarization (and public opinion evaluation of whether it can be overcome) across the world. \nVia @politico: https://t.co/lO…', 'reply_settings': 'everyone', 'referenced_tweets': [{'type': 'retweeted', 'id': '1753054539396075900'}], 'lang': 'en', 'edit_controls': {'edits_remaining': 5, 'is_edit_eligible': False, 'editable_until': datetime.datetime(2024, 3, 8, 20, 17, 1, tzinfo=datetime.timezone.utc)}, 'author_id': '1481596104152125442', 'id': '1766181347406798927', 'edit_history_tweet_ids': ['1766181347406798927'], 'entities': {'mentions': [{'start': 3, 'end': 13, 'username': 'boixserra', 'id': '471649147'}, {'start': 113, 'end': 122, 'username': 'politico', 'id': '9300262'}]}, 'public_metrics': {'retweet_count': 6, 'reply_count': 0, 'like_count': 0, 'quote_count': 0, 'bookmark_count': 0, 'impression_count': 0}, 'conversation_id': '1766181347406798927', 'created_at': '2024-03-08T19:17:01.000Z'}
{'text': 'Polarization (and public opinion evaluation of whether i

In [31]:
includes['tweets']

[<Tweet id=1766181347406798927 text='RT @boixserra: Polarization (and public opinion evaluation of whether it can be overcome) across the world. \nVia @politico: https://t.co/lO…'>,
 <Tweet id=1753054539396075900 text='Polarization (and public opinion evaluation of whether it can be overcome) across the world. \nVia @politico: https://t.co/lOBTMouIbx https://t.co/MVOqCDYIDE'>,
 <Tweet id=1766178225716343208 text='RT @umsi: Spotlight on two new research papers led by UMSI PhD student Joshua Ashkinaze 💡 @JoshAshkinaze\n\nHow AI Ideas Affect the Creativit…'>,
 <Tweet id=1766111764263870608 text='Spotlight on two new research papers led by UMSI PhD student Joshua Ashkinaze 💡 @JoshAshkinaze\n\nHow AI Ideas Affect the Creativity, Diversity, and Evolution of Human Ideas: Evidence From a Large, Dynamic Experiment https://t.co/3bMYfHeI3t \n\nThe Dynamics of (Not) Unfollowing… https://t.co/54ccg1Kdi6'>,
 <Tweet id=1765258076662214795 text='RT @umsi: New research from UMSI:\nThe Dynamics of (Not)

In [40]:
import re

def extract_short_urls(includes):
    urls = []
    url_pattern = r'https://[^\s]+'
    for tweet in includes['tweets']:
        found_urls = re.findall(url_pattern, tweet['text'])
        urls.extend(found_urls)
    return urls
extract_short_urls(includes)

['https://t.co/lO…',
 'https://t.co/lOBTMouIbx',
 'https://t.co/MVOqCDYIDE',
 'https://t.co/3bMYfHeI3t',
 'https://t.co/54ccg1Kdi6',
 'https://t.co/BATenwxakV',
 'https://t.co/3bMYfHffT1']

In [36]:
urls

['https://t.co/lO…',
 'https://t.co/lOBTMouIbx',
 'https://t.co/MVOqCDYIDE',
 'https://t.co/3bMYfHeI3t',
 'https://t.co/54ccg1Kdi6',
 'https://t.co/BATenwxakV',
 'https://t.co/3bMYfHffT1']